# Setup

In [1]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['axes.formatter.useoffset'] = False
import matplotlib.pyplot as plt
import nest
import numpy as np
import os
import re

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils

from matplotlib import rcParams

text_color = 'black' 
rcParams['text.color'] = text_color 
rcParams['axes.labelcolor'] = text_color 
rcParams['xtick.color'] = text_color 
rcParams['ytick.color'] = text_color 
plt.rc('font', size=12)          # controls default text sizes 
plt.rc('axes', titlesize=20)     # fontsize of the axes title 
plt.rc('axes', labelsize=20)    # fontsize of the x and y labels 
plt.rc('xtick', labelsize=20)    # fontsize of the tick labels 
plt.rc('ytick', labelsize=20)    # fontsize of the tick labels 
plt.rc('legend', fontsize=12)    # legend fontsize 
plt.rc('figure', titlesize=26)  # fontsize of the figure title


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 19 2024 15:53:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [ ]:
import warnings
warnings.filterwarnings("ignore", message="ANTLR runtime and generated code versions disagree:.*")

module_name, neuron_model_name, synapse_model_name = NESTCodeGeneratorUtils.generate_code_for(
    nestml_neuron_model= "iaf_cond_alpha.nestml",
    nestml_synapse_model= "stdp_synapse.nestml",
    module_name= "nestml_module",
    post_ports = ["post_spikes", "ca"],
    target_path = "target")


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 19 2024 15:53:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

ANTLR runtime and generated code versions disagree: 4.10!=4.13.0
ANTLR runtime and generated code versions disagree: 4.10!=4.13.0


## Useful Functions

In [ ]:
# Runs the simulation
def simulate(spikes, duration, vm_pre, vm_post, multimeter=None, zoom=False, interval=None):
    # Simulate
    nest.Simulate(duration)
    spikes = sd.get(['events'][0])

    #print(sd.events)
    fig, ax = plt.subplots(3, figsize = (18,10))

    ax[0].plot(vm_pre.events["times"], vm_pre.events["V_m"], color = "b", label = "Pre-synaptic potential")
    ax[0].plot(vm_post.events["times"], vm_post.events["V_m"], color = "orange",  label = "Post-synaptic potential")
    ax[0].set_title("Post-synptic neuron Potential")
    ax[0].set_ylabel("Vm [mV]")
    ax[0].legend()
    if zoom:
        ax[0].set_xlim(interval)

    ax[1].set_title("Rasterplot")
    ax[1].plot( spikes['times'][np.where(spikes['senders']==1)], spikes['senders'][np.where(spikes['senders']==1)], '.', markersize = 40, alpha = 0.5, color = "b", label = "Pre-synaptic spikes")
    ax[1].plot( spikes['times'][np.where(spikes['senders']==2)], spikes['senders'][np.where(spikes['senders']==2)]-1, '.', markersize = 40, alpha = 0.5,color = "orange", label = "Post-synaptic spikes")
    ax[1].legend()
    ax[1].set_ylabel("Neurons")
    ax[1].set_yticklabels("")
    if zoom:
        ax[1].set_xlim(interval)

    ax[2].plot(wr.events["times"], wr.events["weights"], color = 'r')
    ax[2].set_title("STDP")
    ax[2].set_ylabel("Weight [nS]")
    ax[2].set_xlabel("Time [ms]")
    if zoom:
        ax[2].set_xlim(interval)

    # ax[3].plot(multimeter.events["times"], multimeter.events["g_AMPA"], color='blue', label="AMPA conductance")
    # ax[3].plot(multimeter.events["times"], multimeter.events["g_NMDA"], color='green', label="NMDA conductance")
    # ax[3].set_title("Synaptic Conductances")
    # ax[3].set_ylabel("Conductance [nS]")
    # ax[3].set_xlabel("Time [ms]")
    # ax[3].legend()
    # if zoom:
    #     ax[3].set_xlim(interval)

    fig.tight_layout()

# Compares the effects of the STDP assuming the simulation lasts 10s
def compare(vm_pre, vm_post):
    fig, ax = plt.subplots(1, 2, figsize = (18,7))

    ax[0].plot(vm_pre.events["times"], vm_pre.events["V_m"], color = "b", label = "Pre-synaptic potential")
    ax[0].plot(vm_post.events["times"], vm_post.events["V_m"], color = "orange",  label = "Post-synaptic potential")
    ax[0].set_title("First Spike")
    ax[0].set_ylabel("Vm [mV]")
    ax[0].set_xlim([1000,1100])
    ax[0].set_xlabel("Time [ms]")
    ax[0].legend()

    ax[1].plot(vm_pre.events["times"], vm_pre.events["V_m"], color = "b", label = "Pre-synaptic potential")
    ax[1].plot(vm_post.events["times"], vm_post.events["V_m"], color = "orange",  label = "Post-synaptic potential")
    ax[1].set_title("Last Spike")
    ax[1].set_ylabel("Vm [mV]")
    ax[1].set_xlim([9000, 9100])
    ax[1].set_xlabel("Time [ms]")
    ax[1].legend()

    fig.tight_layout()

# Simulation

In [ ]:
#os.environ['LD_LIBRARY_PATH'] = '/tmp/nestml_target:' + os.environ.get('LD_LIBRARY_PATH', '')
nest.ResetKernel()
nest.Install("target/nestml_module")
nest.set_verbosity("M_ERROR")

In [ ]:
print(nest.node_models)
print(nest.synapse_models)

In [ ]:
# Define neuron and synapse models
neuron_model = "iaf_cond_alpha_nestml__with_stdp_nestml"
synapse_model = "stdp_nestml__with_iaf_cond_alpha_nestml"  # Spike-timing-dependent plasticity synapse

# Create two neurons: pre-synaptic and post-synaptic
pre_neuron = nest.Create(neuron_model, 1)
post_neuron = nest.Create(neuron_model, 1)

# Create a weight recorder to log changes in synaptic weights
wr = nest.Create("weight_recorder")
nest.SetDefaults(synapse_model, {"weight_recorder": wr[0]})  # Attach recorder to the STDP synapse

# Display default parameters for the neuron and synapse model and verify connections
print(nest.GetDefaults(neuron_model))
print(nest.GetDefaults(synapse_model))

In [ ]:
# Create the synapse
syn_spec = nest.CollocatedSynapses(
    # {'synapse_model': synapse_model,
    #  'receptor_type': 1}, # General
    {'synapse_model': synapse_model,
     'receptor_type': 2}, # AMPA
    {'synapse_model': synapse_model,
     'receptor_type': 3} # NMDA
)

nest.Connect(pre_neuron, post_neuron, "one_to_one", syn_spec=syn_spec)

print(nest.GetConnections(pre_neuron, post_neuron))

In [ ]:
# Define simulation parameters
duration = 10000  # Simulation time in milliseconds
frequency = 1  # Spike frequency in Hz
delta_t = 10  # Time shift in milliseconds (pre- vs. post-spike)
interval = 100.0 / frequency  # Interval between spikes

# Generate spike times for the spike generator
spike_times = np.arange(interval, duration, interval)
spike_generator = nest.Create("spike_generator", {"spike_times": spike_times})

# Adjust pre- and post-synaptic delays based on delta_t
if delta_t >= 0:
    delta_pre = 0.0
    delta_post = delta_t
else:
    delta_pre = -1 * delta_t
    delta_post = 0.0

# Connect the spike generator to the post-neuron via an NMDA synapse
nest.Connect(spike_generator, post_neuron, "one_to_one", {
    "synapse_model": "static_synapse",
    "weight": 999.0,  # Strong input weight
    "delay": 1.0 + delta_post,  # Adjusted delay
    "receptor_type" : 1 # Directly to V_m
})

# Connect the spike generator to the pre-neuron via an NMDA synapse
nest.Connect(spike_generator, pre_neuron, "one_to_one", {
    "synapse_model": "static_synapse",
    "weight": 999.0,  # Strong input weight
    "delay": 1.0 + delta_pre,  # Adjusted delay
    "receptor_type" : 1 # Directly to V_m
})

# Record spikes using a spike recorder
sd = nest.Create("spike_recorder")
nest.Connect(pre_neuron, sd)  # Connect pre-neuron to spike recorder
nest.Connect(post_neuron, sd)  # Connect post-neuron to spike recorder

# Record membrane potentials using voltmeters
vm_pre = nest.Create("voltmeter")
vm_post = nest.Create("voltmeter")
nest.Connect(vm_pre, pre_neuron)  # Connect voltmeter to pre-neuron
nest.Connect(vm_post, post_neuron)  # Connect voltmeter to post-neuron

# # Record conductances in the postsynaptic neuron
# multimeter = nest.Create("multimeter", params={"record_from": ["g_AMPA", "g_NMDA"]})
# nest.Connect(multimeter, post_neuron)

In [ ]:
# Simulate
simulate(spikes, duration, vm_pre, vm_post, zoom=True, interval=[2000, 2800])

In [ ]:
# Check effects of LTP/LTD    
compare(vm_pre, vm_post)